In [ ]:
# A décommenter à la première exécution si jupyter se plaint de ne pas trouver oracledb
#!pip install --upgrade oracledb

In [5]:
# Compléter ici les imports dont vous avez besoin, ne pas modifier ceux déjà présents
import getpass
from os import getenv
import pandas as pd
import oracledb
import warnings
import matplotlib.pyplot as plt

In [6]:
# Nécessaire pour éviter les problèmes de session
class Connexion(object):
    def __init__(self, login, password):
        self.conn = oracledb.connect(
            user=login,
            password=password,
            host="oracle.iut-orsay.fr",
            port=1521,
            sid="etudom",
        )
        self.conn.autocommit = False

    def __enter__(self):
        self.conn.autocommit = False
        return self.conn

    def __exit__(self, *args):
        self.conn.close()

In [7]:
# La fonction ci-dessous est à utiliser pour exécuter une requête et stocker les résultats dans un dataframe Pandas sans afficher d’alerte.
# Vous pouvez vous en inspirer pour créer vos propres fonctions.
def requete_vers_dataframe(connexion_data, requete, valeurs = None):
    with Connexion(login=connexion_data['login'], password=connexion_data['password']) as connexion:
        warnings.simplefilter(action='ignore', category=UserWarning)
        if valeurs is not None:
            df = pd.read_sql(requete, connexion, params=valeurs)
        else:
            df = pd.read_sql(requete, connexion)
        warnings.simplefilter("always") 
        return df

In [8]:
# Saisir ci-dessous l'édition des JO qui vous a été attribuée. Cela correspond au LibelleHote dans la table HOTE de la base de données
# Par exemple EDITION_JO = "2020 Summer Olympics"
EDITION_JO = "2012 Summer Olympics"
# Saisir ci-dessous le login court de la base utilisée pour votre carnet
SCHEMA = "belhac1"

# TABLEAU DE BORD DES JO

## Partie consultation des données

In [9]:
# On demande à l'utilisateur son login et mot de passe pour pouvoir accéder à la base de données
if getenv("DB_LOGIN") is None:
    login = input("Login : ")
else:
    login = getenv("DB_LOGIN")
if getenv("DB_PASS") is None:
    password = getpass.getpass("Mot de passe : ")
else:
    password = getenv("DB_PASS")
conn = {'login': login, 'password': password}

Login :  belhac1
Mot de passe :  ········


In [10]:
# On vérifie que l'utilisateur est bien connecté à la base de données, que le schéma est bon, et qu'on trouve la bonne édition des JO
data = requete_vers_dataframe(conn, f"SELECT * FROM {SCHEMA}.HOTE WHERE LibelleHote LIKE (:libelle)",{"libelle":EDITION_JO})
id_hote = int(data.IDHOTE.iloc[0])
print(f"Identifiant de l’hôte : {id_hote}")

Identifiant de l’hôte : 54


### Statistiques de base : participation

#### Nombre de nations participant à ces JO

In [38]:
nombreNations = requete_vers_dataframe(conn, f"""select count(distinct codenoc) as "Nombre de nations" from noc n
inner join {SCHEMA}.participation_individuelle pi on pi.noc=n.codenoc
inner join {SCHEMA}.evenement e on pi.idEvent=e.idEvenement
inner join {SCHEMA}.hote h on h.idHote=e.idHote
where h.libellehote='2012 Summer Olympics'""")
nombreNations.iloc[0:1]

,Nombre de nations
0,205


#### Nombre d'athlètes participant à ces JO, répartition par genre et /ou par pays

In [39]:
nombreNations = requete_vers_dataframe(conn, f"""select a.Genre,n.codenoc,count(distinct a.idathlete) as "Nombre d'Athlètes" from noc n
inner join {SCHEMA}.participation_individuelle pi on pi.noc=n.codenoc
inner join {SCHEMA}.evenement e on pi.idEvent=e.idEvenement
inner join {SCHEMA}.hote h on h.idHote=e.idHote
inner join {SCHEMA}.athlete a on a.idAthlete=pi.idAthlete
where h.libellehote='2012 Summer Olympics'
group by a.Genre,n.codenoc
order by count(distinct a.idathlete) DESC""")
nombreNations.iloc[0:10]

,GENRE,CODENOC,Nombre d'Athlètes
0,Male,USA,173
1,Female,USA,148
2,Male,RUS,147
3,Female,RUS,135
4,Male,GBR,131
5,Male,FRA,122
6,Female,GBR,119
7,Male,GER,119
8,Female,CHN,116
9,Male,AUS,111


### Les médailles décernées lors de ces JO

#### Médailles individuelles (épreuves individuelles)

##### Tableau du TOP 10 individuel

In [40]:
medailles_athletes_indiv = requete_vers_dataframe(conn, f"""SELECT IDATHLETE, NOMATHLETE, PRENOMATHLETE, medaillesOr, medaillesArgent,
          medaillesBronze,
         totalMedailles FROM {SCHEMA}.MEDAILLES_ATH_INDIV WHERE IDHOTE={id_hote}""")
medailles_athletes_indiv.iloc[0:10]

,IDATHLETE,NOMATHLETE,PRENOMATHLETE,MEDAILLESOR,MEDAILLESARGENT,MEDAILLESBRONZE,TOTALMEDAILLES
0,93860,Phelps,Michael,2,1,0,3
1,112216,Yang,Sun,2,1,0,3
2,105512,Bolt,Usain,2,0,0,2
3,115344,Farah,Mo,2,0,0,2
4,126679,Franklin,Missy,2,0,0,2
5,104743,Jong-O,Jin,2,0,0,2
6,112317,Kromowidjojo,Ranomi,2,0,0,2
7,122325,Shiwen,Ye,2,0,0,2
8,103585,Lochte,Ryan,1,1,1,3
9,117285,Campriani,Niccolò,1,1,0,2


##### Répartition par NOC

In [10]:
medailles_noc_indiv = requete_vers_dataframe(conn, f"""SELECT CODENOC, NOMNOC, medaillesBronze, medaillesArgent, medaillesOr, totalMedailles FROM {SCHEMA}.MEDAILLES_NOC_INDIV WHERE IDHOTE={id_hote}""")
medailles_noc_indiv.iloc[0:10]

,CODENOC,NOMNOC,MEDAILLESBRONZE,MEDAILLESARGENT,MEDAILLESOR,TOTALMEDAILLES
0,USA,United States,23,19,33,75
1,CHN,People's Republic of China,20,26,28,74
2,GBR,Great Britain,13,10,19,42
3,RUS,Russian Federation,21,18,13,52
4,KOR,Republic of Korea,4,7,11,22
5,FRA,France,10,6,9,25
6,JPN,Japan,14,8,7,29
7,IRI,Islamic Republic of Iran,1,5,7,13
8,HUN,Hungary,6,2,6,14
9,ITA,Italy,8,7,5,20


#### Médailles par équipes (épreuves par équipe)

##### Tableau du TOP 10 par équipe

In [11]:
medailles_equipes = requete_vers_dataframe(conn, f"""SELECT IDEQUIPE, equipe, noc, medaillesBronze, medaillesArgent, medaillesOr, totalMedailles FROM {SCHEMA}.MEDAILLES_EQUIPE WHERE IDHOTE={id_hote}""")
medailles_equipes.iloc[0:10]

,IDEQUIPE,EQUIPE,NOC,MEDAILLESBRONZE,MEDAILLESARGENT,MEDAILLESOR,TOTALMEDAILLES
0,24224,"United States - Football, Women",USA,0,0,22,22
1,14677,"Mexico - Football, Men",MEX,0,0,22,22
2,15300,"Netherlands - Hockey, Women",NED,0,0,18,18
3,9201,"Germany - Hockey, Men",GER,0,0,18,18
4,8193,"France - Handball, Men",FRA,0,0,15,15
5,16141,"Norway - Handball, Women",NOR,0,0,15,15
6,24257,"United States - Water Polo, Women",USA,0,0,13,13
7,4472,"Croatia - Water Polo, Men",CRO,0,0,13,13
8,24206,"United States - Basketball, Men",USA,0,0,12,12
9,19051,"Russian Federation - Volleyball, Men",RUS,0,0,12,12


##### Répartition par NOC

In [12]:
medailles_equipes_noc = requete_vers_dataframe(conn, f"""SELECT codenoc, nomnoc, medaillesBronze, medaillesArgent, medaillesOr, totalMedailles FROM {SCHEMA}.MEDAILLES_NOC_EQUIPE WHERE IDHOTE={id_hote}""")
medailles_equipes_noc.iloc[0:10]

,CODENOC,NOMNOC,MEDAILLESBRONZE,MEDAILLESARGENT,MEDAILLESOR,TOTALMEDAILLES
0,USA,United States,24,36,120,180
1,GER,Germany,12,11,44,67
2,RUS,Russian Federation,24,11,31,66
3,GBR,Great Britain,36,21,29,86
4,CHN,People's Republic of China,8,18,29,55
5,MEX,Mexico,0,4,22,26
6,FRA,France,12,24,21,57
7,NED,Netherlands,14,27,18,59
8,AUS,Australia,55,28,15,98
9,NOR,Norway,0,0,15,15


## Partie modification des données

On décide d'ajouter à l'édition des JO étudiée une nouvelle épreuve : le tir à l'arc à poulies. C'est une épreuve de tir à l'arc (*archery*, code `ARC`), qui se fait soit en arc à poulies homme (individuel ou équipe, mais on ne traitera que le cas individuel), soit arc à poulies femmes (individuel ou équipe). Les noms à utiliser pour les événements seront :

- *Compound individual, Men*
- *Compound individual, Women*

Pour simuler les résultats, vous utiliserez les résultats de l'épreuve de [tir à l'arc classique des JO de 2020 en individuel homme](https://www.olympedia.org/results/18000492) en faisant comme s'il s'agissait d'arc à poulies : les athlètes existent déjà dans la base, vous devez maintenant faire en sorte d'ajouter à la base les différents résultats.

Avant de réaliser les modifications, affichez le tableau des médailles individuelles pour l'Italie

In [71]:
# On donne ici l'exemple de comment ajouter un événement. On a stocké dans id_hote l'identifiant des JO sur lesquels on travaille
with Connexion(login=conn['login'], password=conn['password']) as connexion:  # Démarre une nouvelle connexion
    # On démarre une nouvelle transaction
    connexion.begin()
    # On ouvre un curseur, qui est une sorte de pointeur vers une table de résultats
    curseur = connexion.cursor()
    # On passe la requête paramétrée et les paramètres (s'il y en a) au curseur
    curseur.execute(f"SELECT MAX(IDEVENEMENT) FROM {SCHEMA}.EVENEMENT")
    # S'il n'y a pas d'erreur, on peut demander à récupérer les résultats : 
    # - curseur.fetchone() récupère la ligne sur laquelle pointe le curseur, renvoie le résultat sous forme de tuple, et avance à la ligne suivante (si elle existe)
    # - curseur.fetchall() boucle sur toutes les lignes restantes et renvoie une liste de tuples, ou une liste vide s'il n'y a plus de résultats
    # - curseur.fetchmany(nb) renvoie les résultats des nb lignes suivantes sous forme de liste de tuples
    idev = curseur.fetchone()[0]  # Renvoie un tuple, on veut le premier élément du tuple
    idev += 1
    curseur.execute(f"INSERT INTO {SCHEMA}.EVENEMENT VALUES(:idev, :nomev, :statutev, :codedisc, :idhote)", {'idev': idev, 'nomev': 'Compound Individual, Men', 'statutev': 'Olympic', 'codedisc': 'ARC', 'idhote': id_hote})
    connexion.rollback()  # On annule, ce qui est au dessus est pour l'exemple, vous devrez utiliser commit() pour valider la transaction.

In [17]:
medailles_italie_indiv = requete_vers_dataframe(conn, f"""SELECT * FROM {SCHEMA}.MEDAILLES_ATH_INDIV
INNER JOIN {SCHEMA}.NOC N ON N.CODENOC=NOC WHERE NOMNOC='Italy' AND IDHOTE={id_hote}""")
medailles_italie_indiv.iloc[0:10]


,IDATHLETE,NOMATHLETE,PRENOMATHLETE,IDHOTE,NOC,MEDAILLESOR,MEDAILLESARGENT,MEDAILLESBRONZE,TOTALMEDAILLES,CODENOC,NOMNOC
0,117285,Campriani,Niccolò,54,ITA,1,1,0,2,ITA,Italy
1,123942,Di Francisca,Elisa,54,ITA,1,0,0,1,ITA,Italy
2,108200,Molfetta,Carlo,54,ITA,1,0,0,1,ITA,Italy
3,116495,Molmenti,Daniele,54,ITA,1,0,0,1,ITA,Italy
4,123294,Rossi,Jessica,54,ITA,1,0,0,1,ITA,Italy
5,106357,Cammarelle,Roberto,54,ITA,0,1,0,1,ITA,Italy
6,123944,Errigo,Arianna,54,ITA,0,1,0,1,ITA,Italy
7,123293,Fabbrizi,Massimo,54,ITA,0,1,0,1,ITA,Italy
8,111536,Nespoli,Mauro,54,ITA,0,1,0,1,ITA,Italy
9,118242,Occhiuzzi,Diego,54,ITA,0,1,0,1,ITA,Italy


### Qualifications [Bonus]

Le round de qualifications en tir à l'arc à poulie est assez similaire au tir à l'arc classique, avec comme différence qu'il n'y a pas de "X", ils comptent comme des 10 pour départager. En utilisant le tableau des résultats de qualifications, faites en sorte que votre tableau de bord insère les données dans la base puis les affiche.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat des qualifications

### Première manche [Bonus]

Faire de même ici avec les résultats de la première manche. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat de la première manche

### Deuxième manche [Bonus]

Faire de même ici avec les résultats de la deuxième manche. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat de la deuxième manche

### Troisième manche [Bonus]

Faire de même ici avec les résultats de la troisième manche. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat de la troisième manche

### Quarts de finale [Bonus]

Faire de même ici avec les résultats des quarts de finale. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat des quarts de finale

### Demi-finales [Bonus]

Faire de même ici avec les résultats des demies-finales. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat des demies-finales

### Finale [Bonus]

Faire de même ici avec les résultats de la finale. Pour simplifier, on considérera que les scores sont identiques à ceux en arc classique.

In [ ]:
# Faire ici l'insertion des données

In [ ]:
# Faire ici l'affichage du tableau de résultat de la finale

### Résultats des médailles

Pour terminer, saisir les résultats des médailles

In [15]:
with Connexion(login=conn['login'], password=conn['password']) as connexion:  # Démarre une nouvelle connexion
    connexion.begin()
    curseur = connexion.cursor()

    curseur.execute(f"SELECT MAX(IDEVENEMENT) FROM {SCHEMA}.EVENEMENT")
    idEvM = (curseur.fetchone()[0])  # Renvoie un tuple, on veut le premier élément du tuple
    idEvM+=1
    curseur.execute(f"INSERT INTO {SCHEMA}.EVENEMENT VALUES(:idev, :nomev, :statutev, :codedisc, :idhote)", {'idev': idEvM, 'nomev': 'Compound Individual, Men', 'statutev': 'Olympic', 'codedisc': 'ARC', 'idhote': id_hote})
    idEvW=idEvM+1
    curseur.execute(f"INSERT INTO {SCHEMA}.EVENEMENT VALUES(:idev, :nomev, :statutev, :codedisc, :idhote)", {'idev': idEvW, 'nomev': 'Compound Individual, Women', 'statutev': 'Olympic', 'codedisc': 'ARC', 'idhote': id_hote})
    connexion.commit()

In [21]:
data = [
    {"resultat": "1", "nomPrenom": "Mete Gazoz", "medaille": "Gold", "noc": "TUR"},
    {"resultat": "2", "nomPrenom": "Mauro Nespoli", "medaille": "Silver", "noc": "ITA"},
    {"resultat": "3", "nomPrenom": "Takaharu Furukawa", "medaille": "Bronze", "noc": "JPN"},
    {"resultat": "4", "nomPrenom": "Tang Chih-Chun", "medaille": "", "noc": "TPE"},
    {"resultat": "5", "nomPrenom": "Kim Wu-Jin", "medaille": "", "noc": "KOR"},
    {"resultat": "6", "nomPrenom": "Florian Unruh", "medaille": "", "noc": "GER"},
    {"resultat": "7", "nomPrenom": "Brady Ellison", "medaille": "", "noc": "USA"},
    {"resultat": "8", "nomPrenom": "Li Jialun", "medaille": "", "noc": "CHN"},
    {"resultat": "=9", "nomPrenom": "Ilfat Abdullin", "medaille": "", "noc": "KAZ"},
    {"resultat": "=9", "nomPrenom": "Marcus D’Almeida", "medaille": "", "noc": "BRA"},
    {"resultat": "=9", "nomPrenom": "Atanu Das", "medaille": "", "noc": "IND"},
    {"resultat": "=9", "nomPrenom": "Crispin Duenas", "medaille": "", "noc": "CAN"},
    {"resultat": "=9", "nomPrenom": "Khairul Anuar Bin Mohamad", "medaille": "", "noc": "MAS"},
    {"resultat": "=9", "nomPrenom": "Itay Shanny", "medaille": "", "noc": "ISR"},
    {"resultat": "=9", "nomPrenom": "Taylor Worth", "medaille": "", "noc": "AUS"},
    {"resultat": "=9", "nomPrenom": "Jacob Wukie", "medaille": "", "noc": "USA"},
    {"resultat": "=17", "nomPrenom": "Gijs Broeksma", "medaille": "", "noc": "NED"},
    {"resultat": "=17", "nomPrenom": "Jarno De Smedt", "medaille": "", "noc": "BEL"},
    {"resultat": "=17", "nomPrenom": "Denis Gankin", "medaille": "", "noc": "KAZ"},
    {"resultat": "=17", "nomPrenom": "Pravin Jadhav", "medaille": "", "noc": "IND"},
    {"resultat": "=17", "nomPrenom": "Kim Je-Deok", "medaille": "", "noc": "KOR"},
    {"resultat": "=17", "nomPrenom": "Oh Jin-Hyeok", "medaille": "", "noc": "KOR"},
    {"resultat": "=17", "nomPrenom": "Pierre Plihon", "medaille": "", "noc": "FRA"},
    {"resultat": "=17", "nomPrenom": "Tarundeep Rai", "medaille": "", "noc": "IND"},
    {"resultat": "=17", "nomPrenom": "Riau Ega Agata Salsabilla", "medaille": "", "noc": "INA"},
    {"resultat": "=17", "nomPrenom": "Mohammed Ruman Shana", "medaille": "", "noc": "BAN"},
    {"resultat": "=17", "nomPrenom": "Ryan Tyack", "medaille": "", "noc": "AUS"},
    {"resultat": "=17", "nomPrenom": "Sjef van den Berg", "medaille": "", "noc": "NED"},
    {"resultat": "=17", "nomPrenom": "Wang Dapeng", "medaille": "", "noc": "CHN"},
    {"resultat": "=17", "nomPrenom": "Wei Chun-Heng", "medaille": "", "noc": "TPE"},
    {"resultat": "=17", "nomPrenom": "Wei Shaoxuan", "medaille": "", "noc": "CHN"},
    {"resultat": "=17", "nomPrenom": "Steve Wijler", "medaille": "", "noc": "NED"},
    {"resultat": "=33", "nomPrenom": "Andrés Aguilar", "medaille": "", "noc": "CHI"},
    {"resultat": "=33", "nomPrenom": "Luis Álvarez", "medaille": "", "noc": "MEX"},
    {"resultat": "=33", "nomPrenom": "Baatarkhuyag Otgonbold", "medaille": "", "noc": "MGL"},
    {"resultat": "=33", "nomPrenom": "Mátyás Balogh", "medaille": "", "noc": "HUN"},
    {"resultat": "=33", "nomPrenom": "Dave Barnes", "medaille": "", "noc": "AUS"},
    {"resultat": "=33", "nomPrenom": "Galsan Bazarzhapov", "medaille": "", "noc": "ROC"},
    {"resultat": "=33", "nomPrenom": "Daniel Castro", "medaille": "", "noc": "ESP"},
    {"resultat": "=33", "nomPrenom": "Thomas Chirault", "medaille": "", "noc": "FRA"},
    {"resultat": "=33", "nomPrenom": "Nicholas D'Amour", "medaille": "", "noc": "ISV"},
    {"resultat": "=33", "nomPrenom": "Areneo David", "medaille": "", "noc": "MAW"},
    {"resultat": "=33", "nomPrenom": "Deng Yu-Cheng", "medaille": "", "noc": "TPE"},
    {"resultat": "=33", "nomPrenom": "Arif Dwi Pangestu", "medaille": "", "noc": "INA"},
    {"resultat": "=33", "nomPrenom": "Tom Hall", "medaille": "", "noc": "GBR"},
    {"resultat": "=33", "nomPrenom": "Mohamed Hammed", "medaille": "", "noc": "TUN"},
    {"resultat": "=33", "nomPrenom": "Jeff Henckels", "medaille": "", "noc": "LUX"},
    {"resultat": "=33", "nomPrenom": "Oleksiy Hunbin", "medaille": "", "noc": "UKR"},
    {"resultat": "=33", "nomPrenom": "Patrick Huston", "medaille": "", "noc": "GBR"},
    {"resultat": "=33", "nomPrenom": "Yuki Kawata", "medaille": "", "noc": "JPN"},
    {"resultat": "=33", "nomPrenom": "Sanzhar Musayev", "medaille": "", "noc": "KAZ"},
    {"resultat": "=33", "nomPrenom": "Hiroki Muto", "medaille": "", "noc": "JPN"},
    {"resultat": "=33", "nomPrenom": "Sławomir Napłoszek", "medaille": "", "noc": "POL"},
    {"resultat": "=33", "nomPrenom": "Nguyễn Hoàng Phi Vũ", "medaille": "", "noc": "VIE"},
    {"resultat": "=33", "nomPrenom": "Dan Olaru", "medaille": "", "noc": "MDA"},
    {"resultat": "=33", "nomPrenom": "Daniel Pineda", "medaille": "", "noc": "COL"},
    {"resultat": "=33", "nomPrenom": "Alviyanto Prastyadi", "medaille": "", "noc": "INA"},
    {"resultat": "=33", "nomPrenom": "Žiga Ravnikar", "medaille": "", "noc": "SLO"},
    {"resultat": "=33", "nomPrenom": "Youssof Tolba", "medaille": "", "noc": "EGY"},
    {"resultat": "=33", "nomPrenom": "Jean-Charles Valladont", "medaille": "", "noc": "FRA"},
    {"resultat": "=33", "nomPrenom": "Milad Vaziri", "medaille": "", "noc": "IRI"},
    {"resultat": "=33", "nomPrenom": "Antti Vikström", "medaille": "", "noc": "FIN"},
    {"resultat": "=33", "nomPrenom": "Jack Williams", "medaille": "", "noc": "USA"},
    {"resultat": "=33", "nomPrenom": "James Woodgate", "medaille": "", "noc": "GBR"}
]
def determinerMedaille(pos):
    medailles = {1:'Gold', 2:'Silver', 3:'Bronze'}
    position = int(pos.replace('=', ''))
    if position<=3:
        return medailles[position]
    else :
        return ""

In [ ]:
with Connexion(login=conn['login'], password=conn['password']) as connexion:  # Démarre une nouvelle connexion
    connexion.begin()
    curseur = connexion.cursor()
    print(idEvM)
    for resultat in data:
        try:
            curseur.execute(f"SELECT IDATHLETE FROM {SCHEMA}.ATHLETE WHERE PRENOMATHLETE|| ' ' || NOMATHLETE = :nomPrenom", nomPrenom= resultat["nomPrenom"])
            idAthlete = curseur.fetchone()[0]
        except:
            curseur.execute(f"SELECT IDATHLETE FROM {SCHEMA}.ATHLETE WHERE convert(:nomPrenom, 'WE8ISO8859P1', 'US7ASCII') LIKE convert(PRENOMATHLETE || ' '|| NOMATHLETE, 'WE8ISO8859P1', 'US7ASCII')", nomPrenom= resultat["nomPrenom"])
            idAthlete = curseur.fetchone()[0]
        noc = resultat['noc']
        res = resultat['resultat']
        med = determinerMedaille(res)
        curseur.execute(f"INSERT INTO {SCHEMA}.PARTICIPATION_INDIVIDUELLE VALUES(:idA, :idEv, :res, :med, :noc)", {'idA':idAthlete, 'idEv':idEvM, 'res':res, 'med':med, 'noc':noc})
    



    connexion.commit() 
##commit()

## Vérification

Afficher le tableau des médailles de l'Italie après les ajouts de la nouvelle épreuve.

In [24]:
medailles_italie_indiv = requete_vers_dataframe(conn, f"""SELECT * FROM {SCHEMA}.MEDAILLES_ATH_INDIV
INNER JOIN {SCHEMA}.NOC N ON N.CODENOC=NOC WHERE NOMNOC='Italy' AND IDHOTE={id_hote}""")
medailles_italie_indiv.iloc[0:10]


,IDATHLETE,NOMATHLETE,PRENOMATHLETE,IDHOTE,NOC,MEDAILLESOR,MEDAILLESARGENT,MEDAILLESBRONZE,TOTALMEDAILLES,CODENOC,NOMNOC
0,117285,Campriani,Niccolò,54,ITA,1,1,0,2,ITA,Italy
1,123942,Di Francisca,Elisa,54,ITA,1,0,0,1,ITA,Italy
2,108200,Molfetta,Carlo,54,ITA,1,0,0,1,ITA,Italy
3,116495,Molmenti,Daniele,54,ITA,1,0,0,1,ITA,Italy
4,123294,Rossi,Jessica,54,ITA,1,0,0,1,ITA,Italy
5,106357,Cammarelle,Roberto,54,ITA,0,1,0,1,ITA,Italy
6,123944,Errigo,Arianna,54,ITA,0,1,0,1,ITA,Italy
7,123293,Fabbrizi,Massimo,54,ITA,0,1,0,1,ITA,Italy
8,111536,Nespoli,Mauro,54,ITA,0,1,0,1,ITA,Italy
9,118242,Occhiuzzi,Diego,54,ITA,0,1,0,1,ITA,Italy


### Nettoyage

Supprimez maintenant tous les ajouts précédents (médailles, résultats détaillés, nouvelles épreuves) de votre base.

In [26]:
with Connexion(login=conn['login'], password=conn['password']) as connexion:  # Démarre une nouvelle connexion
    connexion.begin()
    curseur = connexion.cursor()
    curseur.execute(f"""DELETE FROM PARTICIPATION_INDIVIDUELLE WHERE IDEVENT={idEvM}""")

    curseur.execute(f"""DELETE FROM EVENEMENT WHERE IDEVENEMENT={idEvM}""")
    curseur.execute(f"""DELETE FROM EVENEMENT WHERE IDEVENEMENT={idEvW}""")
    

    connexion.commit() 
##commit()